In [ ]:
%pip install tqdm
%pip install numpy
%pip install sentencepiece protobuf
%pip install python-dotenv==1.2.1
%pip install openai==2.16.0
%pip install azure-ai-inference==1.0.0b9
%pip install sentence-transformers==5.2.2 tiktoken==0.12.0

%restart_python

In [ ]:
import os
import json
import asyncio
import numpy  as np
import scipy  as sp
from dotenv        import load_dotenv
from logging       import Logger, getLogger
from openai        import OpenAI, AsyncOpenAI
from tqdm.notebook import tqdm
from tqdm.asyncio  import tqdm_asyncio

import pandas as pd
from pyspark.sql import SparkSession
from delta       import configure_spark_with_delta_pip

from azure.ai.inference.models import SystemMessage, UserMessage
from sentence_transformers     import SentenceTransformer


In [ ]:
builder = SparkSession.builder\
            .config("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")\
            .config("spark.sql.parquet.output.committer.class", "org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter")\
            .config("spark.mapreduce.fileoutputcommitter.marksuccessfuljobs","false")\
            .config("spark.sql.adaptive.enabled", True)\
            .config("spark.sql.shuffle.partitions", "auto")\
            .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "100MB")\
            .config("spark.sql.adaptive.coalescePartitions.enabled", True)\
            .config("spark.sql.dynamicPartitionPruning.enabled", True)\
            .config("spark.sql.autoBroadcastJoinThreshold", "10MB")\
            .config("spark.sql.session.timeZone", "Asia/Tokyo")\
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .config("spark.databricks.delta.write.isolationLevel", "SnapshotIsolation")\
            .config("spark.databricks.delta.optimizeWrite.enabled", True)\
            .config("spark.databricks.delta.autoCompact.enabled", True)
            # Delta Lake 用の SQL コミットプロトコルを指定
            # Parquet 出力時のコミッタークラスを指定
            # Azure Blob Storage (ABFS) 用のコミッターファクトリを指定
            # '_SUCCESS'で始まるファイルを書き込まないように設定
            # AQE(Adaptive Query Execution)の有効化
            # パーティション数を自動で調整するように設定
            # シャッフル後の1パーティションあたりの最小サイズを指定
            # AQEのパーティション合成の有効化
            # 動的パーティションプルーニングの有効化
            # 小さいテーブルのブロードキャスト結合への自動変換をするための閾値調整
            # SparkSessionのタイムゾーンを日本標準時刻に設定
            # Delta Lake固有のSQL構文や解析機能を拡張モジュールとして有効化
            # SparkカタログをDeltaLakeカタログへ変更
            # Delta Lake書き込み時のアイソレーションレベルを「スナップショット分離」に設定
            # 書き込み時にデータシャッフルを行い、大きなファイルを生成する機能の有効化
            # 書き込み後に小さなファイルを自動で統合する機能の有効化


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
# .env ファイルを読み込む
load_dotenv()

# 環境変数の取得
AI_FOUNDRY_ENDPOINT = os.environ.get("AI_FOUNDRY_ENDPOINT")
AI_FOUNDRY_API_KEY  = os.environ.get("AI_FOUNDRY_API_KEY")
AI_FOUNDRY_MODEL    = os.environ.get("AI_FOUNDRY_MODEL")
MAX_TOKENS          = os.environ.get("MAX_TOKENS")
TEMPERATURE         = os.environ.get("TEMPERATURE")
TOP_P               = os.environ.get("TOP_P")

# メモ：
# ウィジット経由でのパラメータの取得方法では、無条件にパラメータの型がStringに変換されてしまう
# そのため、受け取ったパラメータを適切に型変換する必要がある
MAX_TOKENS  = int(MAX_TOKENS)
TEMPERATURE = float(TEMPERATURE)
TOP_P       = float(TOP_P)


# 簡易デバッグ用
print(f'AI_FOUNDRY_ENDPOINT: {AI_FOUNDRY_ENDPOINT}')
print(f'AI_FOUNDRY_API_KEY:  {AI_FOUNDRY_API_KEY}')
print(f'AI_FOUNDRY_MODEL:    {AI_FOUNDRY_MODEL}')
print(f'MAX_TOKENS:          {MAX_TOKENS}')
print(f'TEMPERATURE:         {TEMPERATURE}')
print(f'TOP_P:               {TOP_P}')

In [ ]:
# メモ：
# cohort.npzは以下のようなデータ構成になっている
# cohort.npz
#     |-- data              : 計算済みコホート係数行列
#     |-- indices           : データの位置指定子(列)
#     |-- indptr            : 行ごとのスライスインデックス
#     |-- shape             : コホート係数行列の形(ADIDのリスト × コホートキャプションのリスト)
#     |-- adid_list         : ADIDのリスト(列)
#     |-- business_codelist : コホートキャプションIDのリスト(行)
TARGET      = "/Volumes/stgadintedmpadintedi/featurestore/behaviorvector/cohort.npz"
npz         = np.load(TARGET, allow_pickle=True)
np_codelist = npz["business_codelist"]


# CODELISTに対するキャプションリストを取得
NAVIT_BUSINESS_TABLE = "adinte_datainfrastructure.list.navit_business"
sdf_navit_business   = spark.read.table(NAVIT_BUSINESS_TABLE)\
							.select(['BUSINESS_CODE', 'BUSINESS_NAME_S'])\
                            .join(spark.createDataFrame([(elem,) for elem in np_codelist.tolist()], ['BUSINESS_CODE']), on='BUSINESS_CODE', how='inner')

sdf_navit_business.toPandas().to_csv('cohort_caption_list.csv', index=False, header=True)
pdf_navit_business   = pd.read_csv('cohort_caption_list.csv', header=0)
dict_code2name       = pdf_navit_business.set_index('BUSINESS_CODE')['BUSINESS_NAME_S'].to_dict()
dict_code2name

In [ ]:
llmClient  = AsyncOpenAI(base_url=AI_FOUNDRY_ENDPOINT, api_key=AI_FOUNDRY_API_KEY)
system_msg = SystemMessage(content=(
				"あなたは、特定の場所・施設名から、そこに集まる人々の輪郭を鮮明に描き出す高度な空間データアナリストです。\n"
                "それぞれの場所に対し、その空間の特性、利用者の属性、行動パターンを深掘りし、以下のJSON形式で分析結果を出力してください。\n\n"

				"【出力形式（厳守）】\n"
				"回答は必ず以下のJSON形式のみを出力してください。\n"
				"各場所・施設名をキーとし、150文字以内の短文としてまとめてください。\n"
				"Markdown記法（```json 等）は含めず、生のJSONテキストのみを返してください。\n\n"
                """
				{
					"入力された場所・施設名1": ["具体的なペルソナと行動1（属性＋場所＋行為）", "具体的なペルソナと行動2（属性＋場所＋行為）", ...],
					"入力された場所・施設名2": ["具体的なペルソナと行動3（属性＋場所＋行為）", ...],
				}
				"""
				"\n\n"
				
				"【抽出・生成のルール】\n"
				"1. 語彙の解像度（1文120文字程度を推奨）\n"
                "- 単語の羅列ではなく、情景が浮かぶ「属性＋具体的な状況＋行動」のセットで記述してください。\n"
				"- NG例: 「公園」「運動」「サラリーマン」\n"
				"- OK例: 「週末の親水公園で大型犬を遊ばせる愛犬家」「深夜の24時間ジムで大会に向けて追い込むトレーニー」「早朝の駅ビルでPC作業をするノマドワーカー」\n"
				"2. 出力ボリューム\n"
				"- 入力された場所・施設名、7つずつ抽出してください。\n\n"
			))

async def fetch_analysis(semaphore:asyncio.Semaphore, user_msg:str):
	for idx in range(10):
		idx_temperature = TEMPERATURE * (0.9 ** idx)
		try:
			async with semaphore:
				response     = await llmClient.chat.completions.create(
										messages=[system_msg, user_msg],
										tools=None,
										tool_choice=None,
										max_tokens=MAX_TOKENS,
										temperature=idx_temperature,
										top_p=TOP_P,
										model=AI_FOUNDRY_MODEL
									)
				raw_content  = response.choices[0].message.content

			content_dict = json.loads(raw_content)
			return content_dict
			
		except json.JSONDecodeError as e:
			print(f"[Attempt {idx+1}] Decode Error:: {e}")
			print(raw_content)
			print()

			# 一旦、サーバー負荷の軽減しつつリトライ
			await asyncio.sleep(1)
		
		except Exception as e:
			print(f"[Attempt {idx+1}] API Error: {e}")

			# 一旦、サーバー負荷の軽減しつつリトライ
			await asyncio.sleep(3)
	
	print(f"Failed all 10 attempts for this batch.")
	return {}

async def main():
    semaphore = asyncio.Semaphore(10)
    tasks     = []
    for idx in range(0, len(data_list), 2):
        pair_place = data_list[idx : idx+2]
        user_msg   = UserMessage(content=json.dumps(pair_place))
        tasks.append(fetch_analysis(semaphore, user_msg))
    
    results       = await tqdm_asyncio.gather(*tasks)
    analysis_data = {}
    for result in results:
        analysis_data |= result
    
    return analysis_data

data_list     = [dict_code2name[int(elem)] for elem in np_codelist.tolist()]
analysis_data = await main()
analysis_data

In [ ]:
model            = SentenceTransformer('pkshatech/GLuCoSE-base-ja')
count_list       = [len(analysis_data[key]) for key in analysis_data]
list_location    = [scene                   for key in analysis_data for scene in analysis_data[key]]
np_matrix        = model.encode(list_location)                     # (スポット数N × シーン数M) × 768


indices      = np.cumsum(count_list)[:-1]
final_matrix = np.array([chunk.mean(axis=0) for chunk in np.split(np_matrix, indices)])
final_matrix